# Install necessary packages to run the code

In [1]:
#  Install newspaper3k package used in web scraping and corpus building
!pip install newspaper3k

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Install emoji package
!pip install --upgrade emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Autheticating using Twitter API key and Getting Twitter User data

In [3]:
# importing libraries
import pandas as pd
import numpy as np
import time
#to scrape Twitter
import tweepy
from tweepy import OAuthHandler
#warning    
import warnings
warnings.filterwarnings('ignore')

In [4]:
# The Twitter API tokens and keys are set up. To authenticate a user's request and ensure only
# authorized users can access the API, specific keys and tokens are needed.
CONSUMER_KEY = 'TZ7i40WeANa0gWBpytGsjc7se'
CONSUMER_SECRET = 'NRWRlNs01IWFl8xfP6Xsct3gdb8lkOI2kBCNuVMkby47OMeD48'
OAUTH_TOKEN = '2474122806-dWEoJhdMtwoPwFIvnZGQQk2qw5XQScTIjQ6Y92T'
OAUTH_TOKEN_SECRET = '2sLvxi1Zn6MhQr1VJVU8x6Dvjm2EMFIALHXM5vqnZxxCf'
 
# Creates an instance of 'OAuthHandler' class, which is used to authenticate Twitter API
authentication = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET) 
# For the authenticated user, sets the access token and access token secret
authentication.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET) 

# Generates a new instance of the tweepy library's API class
api = tweepy.API(authentication, wait_on_rate_limit = True) 

# Function to get user's tweets

In [5]:
# Function fetchRetweets to get users who retweeted on the top 20 tweets of news channels
# TopNewsChannels is a list of Twitter news channel handles, 
# TweetsCount is the number of tweets for each news channel
def fetchRetweets(topNewsChannels,tweetsCount):
  
# Initializing a list to store user's data
    usersCollection = []

# Iterating over newschannels list 
    for channel in topNewsChannels:
        tweets = api.user_timeline(screen_name=channel, count= tweetsCount)   #fetching number of tweets for a news channel using user_timeline method
# Iterating over each tweet and then fetches retweets of each tweet using get_retweets
        for tweet in tweets:
            retweets = api.get_retweets(tweet.id)
# Iterating over each retweet and creates a user dictionary containing user data from that handle
            for retweet in retweets:
                user = {
                    "Users": retweet.user.screen_name,
                    "FollowersCount": retweet.user.followers_count / max(1, retweet.user.friends_count),
                    "TotalTweets": retweet.user.statuses_count
                }
# Adding dictionary to list
                usersCollection.append(user)

# Returning dataframe with list of dictionaries  
    return pd.DataFrame(usersCollection)

In [6]:
# The news channel twitter handles tags for extraction of tweets and users
ChannelHandles = ["FoxNews", "nytimes","NBCNews"]
usersOfChannels = fetchRetweets(ChannelHandles,20)
# Printing the fetched users
usersOfChannels

,Users,FollowersCount,TotalTweets
0,mallubang77,0.271908,271402
1,JDiaz10957042,0.149606,1898
2,KeiNishikawa3,0.817204,906476
3,DanielaRallis,0.066856,5759
4,Steve_1224,0.964835,50809
...,...,...,...
581,SDK_Resists,1.010965,141399
582,ImmaBioloG2,0.497696,4427
583,dartgunintel,0.250749,255492
584,filafresh,1.642738,2806310


Removing Duplicate Users


In [7]:
# Removing duplicate users, as same user may retweet many times
# Filtering for rows where TotalTweets is greater than 5 and 
# Popularity is greater than 1, resetting the index 
usersOfChannels.drop_duplicates(inplace = True)
# To make sure that all users are authentic let's filter them based on their popularity and their twitter activity.
usersOfChannels = usersOfChannels[(usersOfChannels.TotalTweets > 5) & (usersOfChannels.FollowersCount > 1)]
usersOfChannels = usersOfChannels.reset_index(drop=True)
# The numbe of users is the first value of the tuple
usersOfChannels.shape

(102, 3)

Now we have list of active users from which we can get their information. Following function will extract each user's top 15 tweets


In [8]:
# Function to fetch the most recent 15 tweets of a given user using the Twitter API and returns them as a list.
def fetchUserTweets(user):
# Fetch top 15 tweets of the users fetched earlier
    usertweets = api.user_timeline(screen_name = user, count = 15, tweet_mode = 'extended')
    toptweets = [tweet.retweeted_status.full_text if tweet.full_text.startswith("RT @") else tweet.full_text for tweet in usertweets]
    return toptweets
  

tweets_list = []
# Call fetchUserTweets on user handle from the usersOfChannels column and storing in tweets_list
for user in usersOfChannels['Users']:
    tweets = fetchUserTweets(str(user))
    tweets_list.append(tweets)
# Add UserTweets columns that will contain list of lists of tweets for each user handle
usersOfChannels["UserTweets"] = tweets_list

In [9]:
usersOfChannels

,Users,FollowersCount,TotalTweets,UserTweets
0,BootsieJazz,4.939855,8007,[Always remember that whenever a democrat says...
1,unrulysoljah,1.893738,218527,[I am saddened how some gospel musicians treat...
2,FaithTiffany2,1.093750,54665,"[Our Girl https://t.co/2gYkZ5OaHS, Senator Nan..."
3,FrittsJensen,204.000000,24826,[@ChasingKensho @FoxNews They are literally ta...
4,AustinCCollier1,1.050483,104918,[TURNING THE TABLES: The suspects chose the wr...
...,...,...,...,...
97,CliftonKinnie,4.877106,29633,[These four House Republicans voted against th...
98,yeskyyes,41.449612,14948,[High-Tech Neocannibalism\n高科技新食人主义 https://t....
99,MOMO198425,2.975610,10705,"[@laomanshujuku 好像还有福建, 更正：：理论上央行可以追踪数字货币（不是现金..."
100,SDK_Resists,1.010965,141399,[Speaker McCarthy’s debt limit proposal would ...


# Preprocessing tweets 

In [10]:
# Importing neccesary libraries for preprocessing the tweets of the each user
import spacy 
import re
import string
from spacy.lang.en.stop_words import STOP_WORDS

# Loads the small english language model called spacy while also creating a set of all the words present in the vocabulary of the model
nlp = spacy.load("en_core_web_sm") 
stop = set(STOP_WORDS) # Creates a set of stop words
exclude = set(string.punctuation) # Creates a set of unctuation marks imported from the "string" module
words = set(nlp.vocab.strings)

In [11]:
# Defined a function called "preprocessing" which takes in the users tweets as the argument and performs several text preprocessing steps on them 

def preprocessing(usertweets):
    # Clean of tweets
    cleanedData = []
    for usertweet in usertweets:
        tweet = re.sub('http\S+', '', usertweet) # Remove links
        tweet = re.sub('RT', '', tweet) # Remove RT of retweet
        tweet = re.sub('@[^\s]+','',tweet) # Remove usernames 
        tweet = "".join([char for char in tweet if char not in string.punctuation]) # Remove punctuations
        tweet = tweet.lower() # Converting to lowercase letters
        tweet = ' '.join([word for word in tweet.split() if word not in (stop)]) # Removing stop words
        tweet = ' '.join([word for word in tweet.split() if len(word)>2]) # Removes any word which has less than 2 letters in it
        cleanedData.append(tweet)

    cleanedData = ' '.join(cleanedData) # Joining all tweets

    # Tokenization
    processedTweets = nlp(cleanedData) 

    # Stemming (lemmatization in spacy)
    processedTweets = [token.lemma_ for token in processedTweets] # Apply the lemmetization

    processedTweets = [word for word in processedTweets if len(word)>2] # Filter out the words which has less than 2 letters in it

    processedTweets = ' '.join(w for w in processedTweets if w in words) # Filter out the words which are not present in the spacy vocabulary

    return processedTweets # Return the processes tweets of a user


In [12]:
# Apply the preprocessing func to each row in the "UserTweets" column and creating a new column "ptweets" to store the resultant preprocessed tweets
usersOfChannels["ptweets"] = usersOfChannels['UserTweets'].apply(lambda x : preprocessing(x))


In [13]:
# Print out the resultant processed data
usersOfChannels

,Users,FollowersCount,TotalTweets,UserTweets,ptweets
0,BootsieJazz,4.939855,8007,[Always remember that whenever a democrat says...,remember democrat say exact opposite true ford...
1,unrulysoljah,1.893738,218527,[I am saddened how some gospel musicians treat...,gospel musician treat house god justifies not ...
2,FaithTiffany2,1.093750,54665,"[Our Girl https://t.co/2gYkZ5OaHS, Senator Nan...",girl senator nancy schaefer wages war child pr...
3,FrittsJensen,204.000000,24826,[@ChasingKensho @FoxNews They are literally ta...,literally target disabled child dutch governme...
4,AustinCCollier1,1.050483,104918,[TURNING THE TABLES: The suspects chose the wr...,turn table suspect choose wrong home invade kn...
...,...,...,...,...,...
97,CliftonKinnie,4.877106,29633,[These four House Republicans voted against th...,house republicans vote gop debt limit bill man...
98,yeskyyes,41.449612,14948,[High-Tech Neocannibalism\n高科技新食人主义 https://t....,break argentina announce pay chinese import yu...
99,MOMO198425,2.975610,10705,"[@laomanshujuku 好像还有福建, 更正：：理论上央行可以追踪数字货币（不是现金...",
100,SDK_Resists,1.010965,141399,[Speaker McCarthy’s debt limit proposal would ...,speaker mccarthy debt limit proposal wreck eco...


# Clustering users according to their interests using k-means clustering

In [75]:
# Imported various modules and  functions to perform text clustering analysis and grouping 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans

In [59]:
# To cluster users based on their twitter contents we converted tweets into vectorized form using TF-IDF vectorizer
# The object is configured to exclude words that are too common or too rare and to limit the feature space size. 
# The resulting matrix will be used for text clustering.

# Utilizing TfidfVectorizer, text input can be preprocessed and changed into a numerical format. 
# For each phrase in the corpus, the phrase Frequency-Inverse Document Frequency (TF-IDF) score is calculated. 
text_transform = TfidfVectorizer(max_df=0.9, max_features=200000, min_df=0.1, use_idf=True)
# Builds a vocabulary, and calculates the IDF score for each item in the lexicon.
text_transform.fit(usersOfChannels.ptweets)
# Fits 'ptweets' attribute and transform the text into numerical format matrix
tfidf_matrix = text_transform.transform(usersOfChannels.ptweets)


In [60]:
tfidf_matrix.toarray()

array([[0.        , 0.        , 0.        , ..., 0.10549471, 0.        ,
        0.28816657],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.11162536, ..., 0.07269206, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [61]:
tfidf_matrix.shape

(102, 238)

In [62]:
# Create a dataframe with feature names and user handles and tf-idf scores for each word in each user's messages
pd.DataFrame(tfidf_matrix.toarray(), columns=text_transform.get_feature_names_out(), index = usersOfChannels.Users)

,100,2020,abortion,access,accord,act,age,allegedly,allow,america,...,wednesday,week,white,win,woman,work,world,year,york,you
Users,,,,,,,,,,,,,,,,,,,,,
BootsieJazz,0.0,0.0,0.000000,0.000000,0.000000,0.141144,0.000000,0.15788,0.00000,0.244463,...,0.000000,0.0,0.000000,0.000000,0.360871,0.000000,0.0,0.105495,0.0,0.288167
unrulysoljah,0.0,0.0,0.000000,0.170024,0.000000,0.000000,0.158524,0.00000,0.00000,0.000000,...,0.155166,0.0,0.149005,0.174457,0.000000,0.127529,0.0,0.000000,0.0,0.000000
FaithTiffany2,0.0,0.0,0.111625,0.108789,0.000000,0.000000,0.000000,0.00000,0.00000,0.084225,...,0.000000,0.0,0.000000,0.000000,0.248661,0.000000,0.0,0.072692,0.0,0.000000
FrittsJensen,0.0,0.0,0.000000,0.166354,0.000000,0.000000,0.155102,0.00000,0.00000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
AustinCCollier1,0.0,0.0,0.000000,0.112550,0.000000,0.000000,0.209875,0.11255,0.00000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.075206,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CliftonKinnie,0.0,0.0,0.123388,0.000000,0.112118,0.000000,0.000000,0.00000,0.00000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.091621,0.000000,0.0,0.160704,0.0,0.000000
yeskyyes,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.173927,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
MOMO198425,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000


Using this generated Tf-Idf matrix we can cluster the users.

In [63]:
total_clusters = 3 # Take count to make clusters

# Create a kmeans object and setting the parameter values
kmeans = KMeans(n_clusters=total_clusters, init='k-means++', max_iter=100, n_init=1)

# Fit the kmeans algo to the tf-idf matrix that was generated (grouping the users based on similarity of their tweets data)
kmeans.fit(tfidf_matrix)
# Get cluster labels for each user 
clusters = kmeans.labels_.tolist()

# Add cluster labels in the usersData 
usersOfChannels['Clusters'] = clusters

In [64]:
# Get cluster centers and sorting the cluster centers array in descending order to get the most important features for each cluster first
kmeans.cluster_centers_.argsort()[:, ::-1]

array([[212, 157,  50,  18,  36,  37, 201, 107, 141,  72,  61,  75, 124,
        171,  65,  28,  97,  25, 144,  21, 223, 131, 167,  14, 101,  55,
         81,  47, 163, 122,  34, 114,  11, 116,  31, 185,  98, 147, 170,
         77,  68, 115, 106,  78, 150,  92,  80,  62, 142, 181, 135, 183,
         88, 130,  16, 200, 111, 209, 235, 108, 184, 175, 137, 126, 199,
        186,  85, 206, 233,  94, 218,  22, 154,  67,  35,  15,   0,  56,
          8,   9, 145,  20, 193,  29, 127, 190, 203,  57,  58,  82, 156,
         26, 120, 196,  70, 148,  89, 217,   6, 205, 136, 188, 228, 139,
         13, 173,  99, 232, 121,  66, 198, 187, 230,  63, 216,  23, 220,
         33, 143, 132,  93,   2, 214, 123, 227, 192, 152, 100,  86,  44,
         39, 215, 211, 174, 195,  41, 168,  38, 117,  87, 222,  49,  52,
          7, 182, 177, 146,   1,  45,  64, 210, 219,  24, 158,  46, 133,
        160,  51,   4,  12, 207, 221,  43, 234,  40,  32, 110,  48, 129,
        229, 165, 178, 226, 166,  91, 119, 172, 213

In [65]:
print("Top words per cluster:")
# Get cluster centers and sorting the cluster centers array in descending order to get the most important features for each cluster first
sorted_order = kmeans.cluster_centers_.argsort()[:, ::-1]
# Get list of feature names using get_feature_names_out that were generated in the text_tranform
terms = text_transform.get_feature_names_out()
# Iterate over clusters and printing top 10 terms of each cluster
for i in range(total_clusters):
    print ("Cluster %d:" % i)
    for ind in sorted_order[i, :10]:
        print (' %s' % terms[ind])
    print

Top words per cluster:
Cluster 0:
 trump
 president
 court
 ban
 carroll
 case
 testify
 jury
 new
 federal
Cluster 1:
 biden
 not
 people
 amp
 state
 president
 say
 child
 school
 day
Cluster 2:
 tucker
 carlson
 fox
 vote
 life
 news
 say
 not
 like
 trump


To reduce the dimension of Tf-Idf matrix we define error term, distance matrix using cosine similarity

In [66]:
# Get a distance matrix by subtracting cosine similarity matrix by ones
dist_matrix = 1 - cosine_similarity(tfidf_matrix)
dist_matrix

array([[ 0.00000000e+00,  8.54063278e-01,  6.63652528e-01, ...,
         1.00000000e+00,  8.83852287e-01,  9.90805475e-01],
       [ 8.54063278e-01,  0.00000000e+00,  8.16171223e-01, ...,
         1.00000000e+00,  8.98250459e-01,  8.75759179e-01],
       [ 6.63652528e-01,  8.16171223e-01,  0.00000000e+00, ...,
         1.00000000e+00,  6.40654194e-01,  9.05294756e-01],
       ...,
       [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
       [ 8.83852287e-01,  8.98250459e-01,  6.40654194e-01, ...,
         1.00000000e+00, -2.22044605e-16,  8.83590046e-01],
       [ 9.90805475e-01,  8.75759179e-01,  9.05294756e-01, ...,
         1.00000000e+00,  8.83590046e-01,  0.00000000e+00]])

In [67]:
# Create unique list of names
Uncommon_names = usersOfChannels['Clusters'].unique()
print(Uncommon_names)
# Create a data frame dictionary to store data frames where the keys are unique cluster labels and values are empty dataframes for now
DataFrameDict = {elem : pd.DataFrame for elem in Uncommon_names}

[1 0 2]


In [68]:
print("Dataframe Dictionary: " , DataFrameDict)

Dataframe Dictionary:  {1: <class 'pandas.core.frame.DataFrame'>, 0: <class 'pandas.core.frame.DataFrame'>, 2: <class 'pandas.core.frame.DataFrame'>}


In [69]:
# Iterate over each cluster label
for key in DataFrameDict.keys():
# Assign values in the cluster label in userOfChannels to DataFrameDict
    DataFrameDict[key] = usersOfChannels[:][usersOfChannels.Clusters == key]


In [70]:
DataFrameDict

{1:                Users  FollowersCount  TotalTweets  \
 0        BootsieJazz        4.939855         8007   
 1       unrulysoljah        1.893738       218527   
 2      FaithTiffany2        1.093750        54665   
 3       FrittsJensen      204.000000        24826   
 4    AustinCCollier1        1.050483       104918   
 5           1956kobe        3.437086       113776   
 6          oldriccid        1.130834       110071   
 7            PHD3535        1.094595        36553   
 8       TheJFKReport        9.972566        52262   
 9        AnnStokes55        1.257820       297637   
 11     Engineerbooef       29.166667         5301   
 12      boldlybeaded        1.106354         2924   
 13         intell911        1.221174         2685   
 14      jemnotgem_22        1.043236        97357   
 17           mrddmia        4.101199        82009   
 18    GabrielaBiniek        1.475000        41979   
 19    commonsense258        3.715596       481102   
 20        ryan102857    

In [71]:
# Import libraries for sentiment analysis
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer

In [72]:
# Initialize two dictionaries with uniquenames as keys 
polaritycluster={elem : pd.DataFrame for elem in Uncommon_names}
subjectivitycluster={elem : pd.DataFrame for elem in Uncommon_names}
# Iterate over DataFrameDict for each key
for i in DataFrameDict.keys():
# Join all tweets into a string and returning Textblob object
# Calculate polarity and subjectivity of all tweets in the dataframe
    polaritycluster[i]=TextBlob(' '.join(DataFrameDict[i]['UserTweets'].astype('str'))).sentiment.polarity
    subjectivitycluster[i]=TextBlob(' '.join(DataFrameDict[i]['UserTweets'].astype('str'))).sentiment.subjectivity
# sentiment.polarity - retruns polarity score from -1 to 1
# sentiment.subjectivity - returns subjectivity score from 0 to 1

In [73]:
polaritycluster

{1: 0.06880078595314833, 0: 0.11615838508475312, 2: 0.08377821063631269}

In [74]:
subjectivitycluster

{1: 0.4652149839350785, 0: 0.4681229291690665, 2: 0.4402055801564223}

# Web scraping and corpus building

In [31]:
# Access and modify localization setting of the system
import locale
print(locale.getpreferredencoding())

UTF-8


In [32]:
# Defination for return UTF-8 encoding
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [33]:
# Importing newspaper3k package and import article library
import newspaper
from newspaper import Article

# For similarity measure
from sklearn.metrics.pairwise import cosine_similarity

In [34]:
# Import statements for corpus
import requests
from bs4 import BeautifulSoup
import newspaper
import pandas as pd

# URL of the news papers

bbc_paper = 'https://www.bbc.com/'
cnn_paper='https://edition.cnn.com/'
wp_paper = 'https://washingtonpost.com'
# Make a request to the website and get the HTML content

# bbc Post:
bbc_response = requests.get(bbc_paper)
bbc_html_content = bbc_response.content

# CNN:
cnn_response = requests.get(cnn_paper)
cnn_html_content = cnn_response.content

# wp:
wp_response = requests.get(wp_paper)
wp_html_content = wp_response.content

# Use BeautifulSoup to parse the HTML content
bbc_soup = BeautifulSoup(bbc_html_content, 'html.parser')
cnn_soup = BeautifulSoup(cnn_html_content, 'html.parser')
wp_soup = BeautifulSoup(wp_html_content, 'html.parser')

# Find all the links on the page
bbc_links = bbc_soup.find_all('a')
cnn_links = cnn_soup.find_all('a')
wp_links = wp_soup.find_all('a')


# The more the links and the larger the corpus the better and more specific the more precise the recommendation will be


# Filter the links to only include articles from the Washington Post
bb_links = [link for link in bbc_links if 'www.bbc.com' in link.get('href', '')]
cn_links = [link for link in cnn_links if 'edition.cnn.com' in link.get('href','')]
wp_links = [link for link in wp_links if 'www.washingtonpost.com' in link.get('href','')]

# Create a list to hold the article data
article_data = []

# Loop through the links and get the article text and link for each one
for link in bb_links:
    # Use newspaper to download and parse the article
    article = newspaper.Article(link['href'])
    article.download()
    article.parse()
    
    # Add the article data to the list
    article_data.append({
        'link': link['href'],
        'text': article.text
    })

# cnn
for link in cn_links:
    # Use newspaper to download and parse the article
    article = newspaper.Article(link['href'])
    article.download()
    article.parse()
    
    # Add the article data to the list
    article_data.append({
        'link': link['href'],
        'text': article.text
    })

# wp
for link in wp_links:
    # Use newspaper to download and parse the article
    article = newspaper.Article(link['href'])
    article.download()
    article.parse()
    
    # Add the article data to the list
    article_data.append({
        'link': link['href'],
        'text': article.text
    })

# Create a Pandas DataFrame from the article data
df = pd.DataFrame(article_data)

# Print the DataFrame
print(df)


                                                  link  \
0                                 https://www.bbc.com/   
1                             https://www.bbc.com/news   
2                            https://www.bbc.com/sport   
3                             https://www.bbc.com/reel   
4                         https://www.bbc.com/worklife   
..                                                 ...   
507                     https://www.washingtonpost.com   
508     https://www.washingtonpost.com/about-the-post/   
509  https://www.washingtonpost.com/anonymous-news-...   
510  https://www.washingtonpost.com/discussions/202...   
511      https://www.washingtonpost.com/cookie-policy/   

                                                  text  
0    Pep Guardiola says Manchester City are deliver...  
1    Trump rape accuser E Jean Carroll takes the st...  
2    The Ironman unbroken by a bomb\n\nLosing blood...  
3               Why your life is probably a simulation  
4                 

In [35]:
#   Dropping the duplicate links from the dataframe
df.drop_duplicates(inplace = True)

In [36]:
# Dataframe containing data extracted from the different news channles
df

,link,text
0,https://www.bbc.com/,Pep Guardiola says Manchester City are deliver...
1,https://www.bbc.com/news,Trump rape accuser E Jean Carroll takes the st...
2,https://www.bbc.com/sport,The Ironman unbroken by a bomb\n\nLosing blood...
3,https://www.bbc.com/reel,Why your life is probably a simulation
4,https://www.bbc.com/worklife,How we think
...,...,...
502,https://www.washingtonpost.com/privacy-policy/,Site Information Privacy Policy\n\nShare\n\nPu...
503,https://www.washingtonpost.com/cookie-policy/,Ask the Post Cookie Notice\n\nShare\n\nUpdated...
504,https://www.washingtonpost.com/discussions/202...,Site Information RSS Terms of Service\n\nShare...
505,https://www.washingtonpost.com/information/202...,Site Information Ad choices\n\nShare\n\nPublis...


Preprocessing of the articles and the texts to for further processing

In [37]:
def processArticles(articles):
  #cleaning of articles
  cleanedarticles = []
  for article in articles: 
      article = re.sub("[^a-zA-Z]"," ", str(article))
      article = article.lower() # Converting to lowercase letters
      article = ' '.join([word for word in article.split() if word not in (stop)]) # Removing stop words
      article = ' '.join([word for word in article.split() if len(word)>2])   

      # Tokenization
      article = nlp(article) 

      # Stemming (lemmatization in spacy)
      article = [token.lemma_ for token in article]

      # Remove small and unnecessary words
      article = [word for word in article if len(word)>2]
      article = ' '.join(w for w in article if w in words)

      cleanedarticles.append(article)
#  Return the processed article text
  return cleanedarticles

In [38]:
df['text'] = processArticles(df['text'])

In [39]:
# The dataframe is a corpus of the web scrapped data and will be further used in recommendation system and similarity matching
df

,link,text
0,https://www.bbc.com/,pep say manchester city delivering option win ...
1,https://www.bbc.com/news,trump rape accuser jean carroll take stand emo...
2,https://www.bbc.com/sport,unbroken bomb lose blood run low time know cho...
3,https://www.bbc.com/reel,life probably simulation
4,https://www.bbc.com/worklife,think
...,...,...
502,https://www.washingtonpost.com/privacy-policy/,site information privacy policy share publish ...
503,https://www.washingtonpost.com/cookie-policy/,ask post cookie notice share update february c...
504,https://www.washingtonpost.com/discussions/202...,site information term service share encourage ...
505,https://www.washingtonpost.com/information/202...,site information choice share publish septembe...


# Sentiment Analysis

Doing sentiment analysis of a users tweets to understand his positive, negative and netural sentiments towards different topics in the tweets

In [40]:
# Fetch the latest tweets and retweets of a user for news using pagination which helps in retriving more than 200 tweets per user in batches or pages

# function to fetch tweets and retweets
def fetchTweets(user, count=1000):
    all_tweets = []
    max_id = None
    while True:
        # Fetch the latest batch of tweets using the max_id parameter
        usertweets = api.user_timeline(screen_name=user, count=count, tweet_mode='extended', max_id=max_id)
        if len(usertweets) == 0:
            break
        all_tweets.extend(usertweets)
        max_id = usertweets[-1].id - 1
    # Retweets filtering 
    toptweets = [tweet.retweeted_status.full_text if tweet.full_text.startswith("RT @") else tweet.full_text for tweet in all_tweets]
    return all_tweets


In [41]:
# Storing the tweets in twes list fo the top contributor user in our news users channel dataframe
twes = fetchTweets(str(usersOfChannels['Users'][1]))

In [42]:
# Total number of tweets retrieved of the user handle
len(twes)

3250

In [43]:
# Import the emoji library
import emoji

# Define a function to check if a tweet contains an emoji
def contains_emoji(tweet):
    for character in tweet:
        if character in emoji.EMOJI_DATA:
            return True
    return False

new = []
# Filter the tweets to remove the ones with only the emoji text for better processing of the data during sentiment analysis
for item in twes:
    if isinstance(item.full_text, str) and contains_emoji(item.full_text) == False:
        # The full_text attribute is a string
        new.append([item.full_text])


In [44]:
# Number of tweets after removing non text tweets
len(new)

1786

In [45]:
# Function for preprocessing of the tweet text
def preprocess_text(tweet):
    # Cleaning of tweets
    cleanedData = []
    tweet = re.sub('http\S+', '', tweet) # Remove links
    tweet = re.sub('RT', '', tweet) # Remove RT of retweet
    tweet = re.sub('@[^\s]+','',tweet) # Remove usernames 
    tweet = "".join([char for char in tweet if char not in string.punctuation]) # Remove punctuations
    tweet = tweet.lower() # Converte to lowercase letters
    tweet = ' '.join([word for word in tweet.split() if word not in (stop)]) # Removing stop words
    tweet = ' '.join([word for word in tweet.split() if len(word)>2]) #  Removes any word which has less than 2 letters in it

    # # Tokenization
    # processedTweets = nlp(tweet) 

    # # Stemming (lemmatization in spacy)
    # processedTweets = [token.lemma_ for token in processedTweets]  # Applying the lemmetization

    # processedTweets = [word for word in processedTweets if len(word)>2] # Filter out the words which has less than 2 letters in it

    # processedTweets = ' '.join(w for w in processedTweets if w in words) # Filter out the words which are not present in the spacy vocabulary

    return tweet

Perform sentiment analysis using VADER lexicon library and using hybrid deep neural network ANN-LSTM model to train, fit, test and evaluate the performance of our classification model

In [46]:
#  Imports for neural network and lstm and sentiment analyzer
import pandas as pd
import numpy as np
import nltk
import re
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
#from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
df1 = pd.DataFrame(new, columns=['text'])
df1['ptext'] = None
# Preprocess the tweet text
for i,text in enumerate(df1['text']):
  df1['ptext'][i] = preprocess_text(text)
for i,t in enumerate(df1['ptext']):
  if len(t) == 0:
    df1['ptext'][i] = "world news"

# VADER lexicon sentiment analyzer library to caluclate the polarity of the sentiment
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
df1['sia_sentiment'] = df1['ptext'].apply(lambda x: sia.polarity_scores(x)['compound'])

# Assign binary labels based on threshold value of the polarity
df1['sia_sentiment_labels'] = df1['sia_sentiment'].apply(lambda x: 1 if x >= 0.7 else 0)

# Encode the target variable
le = LabelEncoder()
y = le.fit_transform(df1['sia_sentiment_labels'])

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df1['ptext'], y, test_size=0.2, random_state=10)

# Tokenize the text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# Pad the sequences to the same length
max_len = 50
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

kernel_size = 3
filters = 250
# Build the LSTM model
model = Sequential()
model.add(Embedding(5000, 128, input_length=max_len))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model and use early stop for the process to stop from running for a long time loop and call back 
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model on the test set
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Epoch 1/10
36/36 [==============================] - 8s 135ms/step - loss: 0.3072 - accuracy: 0.9264 - val_loss: 0.2041 - val_accuracy: 0.9476
Epoch 2/10
36/36 [==============================] - 7s 188ms/step - loss: 0.2255 - accuracy: 0.9431 - val_loss: 0.2006 - val_accuracy: 0.9476
Epoch 3/10
36/36 [==============================] - 4s 119ms/step - loss: 0.1930 - accuracy: 0.9431 - val_loss: 0.1903 - val_accuracy: 0.9476
Epoch 4/10
36/36 [==============================] - 4s 118ms/step - loss: 0.0795 - accuracy: 0.9772 - val_loss: 0.2156 - val_accuracy: 0.9476
Epoch 5/10
36/36 [==============================] - 6s 166ms/step - loss: 0.0256 - accuracy: 0.9930 - val_loss: 0.2504 - val_accuracy: 0.9476
Epoch 6/10
36/36 [==============================] - 4s 115ms/step - loss: 0.0091 - accuracy: 0.9982 - val_loss: 0.2527 - val_accuracy: 0.9371
Epoch 6: early stopping
Test loss: 0.22072049975395203
Test accuracy: 0.9441340565681458


We see that the test accuracy of the classification model is 0.95, which is very high implying that the VADER lexicon analyzer has a good accuracy in sentiment polarity prediction

In [47]:
# Remove the duplicate links
df1.drop_duplicates(inplace = True)

# Recommendation system using sentiment analysis results

Based on the sentiments of the users tweets now we will collect the tweets with the positive sentiment and recommend the users articles form the web scraped corpus to pertain the interest of the user in reading more of the articles and engage them on the social media platform or gain increment in product sales

In [48]:
#  Reset the index of the dataframe to remove missing values after sentiment analysis
df1 = df1.reset_index(drop=True)

In [49]:
# Collect the preprocesses positve tweets for VADER analyzer
sent_recom_preprocessed_tweets = []
for t in range(len(df1)):
  if df1['sia_sentiment_labels'][t] == 1:
    sent_recom_preprocessed_tweets.append(df1['ptext'][t])



# Find similar articles
# Import tfidf vectorizer and cosine similarity library
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Use TF-IDF vectorizer to transform article text and preprocessed tweets into feature vectors
vectorizer = TfidfVectorizer()
# Fit transform corpus text
article_features = vectorizer.fit_transform(df['text'])
# Fit transorm VADER text
sent_recom_tweet_features = vectorizer.transform(sent_recom_preprocessed_tweets)


# Compute cosine similarities between tweet features and article features
sent_recom_similarities = cosine_similarity(sent_recom_tweet_features, article_features)

# Get indices of top 200 articles most similar to the user's tweets
sent_recom_top_article_indices = sent_recom_similarities.mean(axis=0).argsort()[::-1][:246]


# Get recommended articles
sentiment_recommended_user_articles = df.iloc[sent_recom_top_article_indices][['text', 'link']]


In [50]:
# Top recommended articles and links for the user based on sentiment polarity
sentiment_recommended_user_articles[:10]

,text,link
118,listen min comment story comment gift article ...,https://www.washingtonpost.com/opinions/2023/0...
231,listen min comment story comment gift article ...,https://www.washingtonpost.com/politics/2023/0...
358,listen min comment story comment gift article ...,https://www.washingtonpost.com/sports/2023/04/...
115,listen min comment story comment gift article ...,https://www.washingtonpost.com/opinions/2023/0...
200,listen min comment story comment gift article ...,https://www.washingtonpost.com/politics/2023/0...
246,listen min comment story comment gift article ...,https://www.washingtonpost.com/advice/2023/04/...
93,listen min comment story comment gift article ...,https://www.washingtonpost.com/business/2023/0...
253,listen min comment story comment gift article ...,https://www.washingtonpost.com/technology/2023...
0,pep say manchester city delivering option win ...,https://www.bbc.com/
111,listen min comment story comment gift article ...,https://www.washingtonpost.com/lifestyle/2023/...


# Recommendation system without sentiment analysis

In [51]:
# Collect the preprocesses positve tweets for Text Blob analyzer
recom_tweets = []
for t in range(len(df1)):
  recom_tweets.append(df1['ptext'][t])

# Fit transform Text Blob text
recom_tweet_features = vectorizer.transform(recom_tweets)

# Compute cosine similarities between tweet features and article features
recom_similarities = cosine_similarity(recom_tweet_features, article_features)

# Get indices of top 200 articles most similar to the user's tweets
recom_top_article_indices = recom_similarities.mean(axis=0).argsort()[::-1][:190]

# Get recommended articles
simple_recommended_user_articles = df.iloc[recom_top_article_indices][['text', 'link']]

In [52]:
# # Top recommended articles without sentiment analysis
simple_recommended_user_articles[10:19]

,text,link
132,listen min comment story comment gift article ...,https://www.washingtonpost.com/opinions/2023/0...
233,john wagner national reporter post break polit...,https://www.washingtonpost.com/people/john-wag...
158,julian business reporter education state unive...,https://www.washingtonpost.com/people/julian-m...
174,listen min comment story comment gift article ...,https://www.washingtonpost.com/media/2023/04/2...
473,hannah sampson reporter focus travel news educ...,https://www.washingtonpost.com/people/hannah-s...
486,site information policy standard washington po...,https://www.washingtonpost.com/policies-and-st...
331,home single woman man buy home mean easy compe...,https://www.washingtonpost.com/home/2023/04/26...
238,listen min comment story comment gift article ...,https://www.washingtonpost.com/politics/2023/0...
118,listen min comment story comment gift article ...,https://www.washingtonpost.com/opinions/2023/0...


In [53]:
# import spacy
# nlp = spacy.load("en_core_web_sm")

# # assume `df` is your dataframe of news articles

# def recommend_articles(tweet_text, df):
#     # preprocess tweet text
#     tweet_doc = nlp(tweet_text.lower())
#     tweet_keywords = [token.lemma_ for token in tweet_doc if not token.is_stop and not token.is_punct and not token.is_space]

#     # search for articles that contain at least one of the tweet keywords in the title or text
#     matching_articles = df[df.apply(lambda x: any(keyword in x['text'].lower() for keyword in tweet_keywords), axis=1)]

#     # return top 5 matching articles
#     return matching_articles.head(10)

In [54]:
# Function defination for jaccard similarity score on the follwoing models
import math
def get_jaccard_sim(set1, set2): 
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    sim = float(len(intersection) / len(union))
    return  float(sim)

In [55]:
#  Store the dataframe 'text' into a single list
twts = []
sent_recom = []
recom = []
for i in df['text']:
  twts.append(i)
for i in sentiment_recommended_user_articles['text']:
  sent_recom.append(i)
for i in simple_recommended_user_articles['text']:
  recom.append(i)

# Result comporision through similarity score

In [56]:
# Jaccard similarity score of the actual articles and the recommended articles using sentiment analysis
get_jaccard_sim(set(twts),set(sent_recom))

0.8116438356164384

In [57]:
# Jaccard similarity score of the actual articles and the recommended articles without sentiment analysis
get_jaccard_sim(set(twts),set(recom))

0.6335616438356164

# Conclusion

The Jaccard similarity is a popular similarity measure in recommendation systems that evaluates how similar two collections of items are to one another. It relies on the hunch that two collections of items that have a lot of overlap might have a connection to one another. The jaccard similarity ranges from 0.0 to 1.0. The higher the jaccard similarity of the two sets, the higher the implication that they are connected to each other.

After running the colab file models multiple times we see that the jaccard similarity score of simple recommendation system is less than recommendation system with sentiment analysis. The sentiment recommendation system is having higher similarity because it is recommending user the articles based on the sentiments of the users which is more close to the actual recommendations provided.